# Python course 6

Submit this notebook to codegrade as python_course_6.ipynb.

Alternatively, copy the function from the notebook into a script and submit as python_course_6.py.

Make sure your notebook actually runs (kernel > restart & run all) without errors before submitting.

Make sure to place the client folder (unzippped) in the same folder as this notebook.
Also make sure you place the option.py file (included in the assignment) in the same folder as the notebook.

In [1]:
from option import Option

For this assignment you can use the functions in the black_scholes.py module, which should be included with this assignment.

In [2]:
import black_scholes as bs

### Dummy data

In [3]:
from client.info_messages import PriceBook, PriceVolume
import datetime

# Stock pricebook
pb_S = PriceBook()
pb_S.instrument_id = 'AAPL'
pb_S.bids = [PriceVolume(99, 100)]
pb_S.asks = [PriceVolume(101, 100)]

# Option pricebook
pb = PriceBook()
pb.instrument_id = 'AAPL_CALL'
pb.bids = [PriceVolume(9, 100)]
pb.asks = [PriceVolume(10, 100)]


call = Option('call', 1, 100)
put = Option('put', 0.5, 110)


S = 100  # Underlying price
r = 0  # Interest rate
sigma = 0.2  # Volatility

import numpy as np
option_feedcode = 'BMW-2020_12_31-140-Call'
expiry = datetime.datetime(2020, 12, 31)
today = datetime.date.today()

def __init__(self, kind, tte, strike):
        self.tte = tte
        self.strike = strike
        self.kind = kind

def __repr__(self):
    return f"[option] kind={str(self.kind)} tte={str(self.tte)}, strike={str(self.strike)}"

def __eq__(self, other):
    if not isinstance(other, self.__class__):
        return NotImplemented
    return self.tte == other.tte and self.strike == other.strike and self.kind == other.kind

from scipy import stats
import numpy as np

_norm_cdf = stats.norm(0, 1).cdf
_norm_pdf = stats.norm(0, 1).pdf


def _d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


def _d2(S, K, T, r, sigma):
    return _d1(S, K, T, r, sigma) - sigma * np.sqrt(T)


def call_value(S, K, T, r, sigma):
    return S * _norm_cdf(_d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * _norm_cdf(_d2(S, K, T, r, sigma))

def put_value(S, K, T, r, sigma):
    return np.exp(-r * T) * K * _norm_cdf(-_d2(S, K, T, r, sigma)) - S * _norm_cdf(-_d1(S, K, T, r, sigma))

def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def best_bid_price(PriceBook):
    
    if PriceBook.bids == []:
        return 
    return sorted(PriceBook.bids, key = lambda x: x.price)[-1].price

def best_ask_price(PriceBook):
    
    if PriceBook.asks == []:
        return 
    return sorted(PriceBook.asks, key = lambda x: x.price)[0].price



In [4]:
call.kind  # call or put
call.tte  # time to expiry (years)
call.strike  # strike price

100

### Parsing option feedcode

Write a function that parses an option feedcode to returns its information.

The feedcode has the following format:
    {instrument_id}-{expiry_date}-{strike}-{put_or_call}

The expiry date has the format: yyyy_mm_dd. The strike will be a integer, and put_or_call will be ```'Put'``` or ```'Call'```.

You should output each part of the info separately. See output below for what you should return.

Example
```python
'BMW-2020_12_31-140-Call'
```
- instrument_id: ```'BMW'```
- expiry_date: ```dt.date(2020, 12, 31)```
- strike: ```140```
- put_or_call: ```'call'```

input: option_feedcode

output: instrument_id (str), expiry_date (dt.date), strike (int), put_or_call (str: ```'put'``` or ```'call'```, all lowercase)

In [5]:
def parse_feedcode(option_feedcode):
    
    list_ = option_feedcode.split('-')
    return (list_[0],
            datetime.datetime.strptime(list_[1],'%Y_%m_%d').date(),
            int(list_[2]),
            list_[3].lower()
    )  
parse_feedcode(option_feedcode)

('BMW', datetime.date(2020, 12, 31), 140, 'call')

### Calculate time to expiry

Write a function that calculates time to expiry (in calendar time and business time) in years given today's date and the date of expiry.

You can assume that the current time is 12:00 (noon) and the option also expires at 12:00 (noon).

The convention for calendar time is that there are 365 days in a year, so the time to expiry in years = number of days to expiry / 365.

The convention for business time is that there are 256 business days in a year, so the time to expiry in years = number of business days to expiry / 256.
For this exercise all days are business days except for Saturdays and Sundays. The expiry will always be a business day but today can be a non-business day.
If today is non-business day you have to account for the half day i.e. if the option expires on Monday and today is the Saturday before, the time to expiry would be 0.5 / 256 as there would only be half a day left.

input: today (dt.date), expiry (dt.date)

output: time_to_expiry_act365 (float), time_to_expiry_bus256 (float)

In [6]:
def calculate_tte(today, expiry):
   
    result = None  
    expiry = expiry.date() if type(expiry) == datetime.datetime else expiry
    today = today.date() if type(today) == datetime.datetime else today
    time_to_exp = expiry - today
    #print(time_to_exp)
    normal_time = time_to_exp.days/365
    #print(normal_time)
    if datetime.date.weekday(today) == 5:      
        exp_days = (expiry - today)
        expiry_in_bd = ((exp_days.days -1.5) / 256) 


    elif datetime.date.weekday(today) == 6:   
        exp_days = ( expiry - today)
        expiry_in_bd = ((exp_days.days -0.5)/ 256) 

    else:
        exp_days = np.busday_count(today,expiry)
        expiry_in_bd = (exp_days / 256)
            
    return normal_time , expiry_in_bd


calculate_tte(today, expiry)

(0.7726027397260274, 0.7890625)


### Option price

Write a function that calculates the black-scholes option price given an option and market data.

input: Option, S, r, sigma

output: option price


In [7]:
def option_price(option, S, r, sigma):
    
    
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        callx = bs.call_value(S, K, T, r, sigma)
        return callx
    
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        putx = bs.put_value(S, K, T, r, sigma)
        return putx

option_price(put, S, r, sigma)

12.21124643357308

### Option opportunity

Write a function that checks a given option PriceBook for arbitrage.

You are given all inputs to calculate the price of the given option. Check the given option pricebook if there is an opportunity to trade on this price.

input: PriceBook, Option, S, r, sigma

output: bool (True if there is a trading opportunity, False otherwise)

In [125]:
def option_opportunity(pb, option, S, r, sigma):
    
    option_price_cp = option_price(option, S, r, sigma)
    value_call = best_ask_price(pb)
    #print(value_call)
    value_put = best_bid_price(pb)
    #print(value_put)
    
    if pb.bids != [] and pb.asks != []:
        if value_call < option_price_cp:
            return True 
        elif value_put > option_price_cp:
            return True
        else:
            return False 
    return False

option_opportunity(pb, call, S, r, sigma)      

True


### Option quotes

Write a function that provides quotes (bid and ask) for the provided option given the PriceBook of the underlying.

A single stock price results in a single option price. Now however, you are given an orderbook instead of a single stock price.
Your function has to determine which price should be used to calculate the option value. The price to be used depends on what action you need to take to hedge.

E.g. If the option is a call:
    If you buy a call, you hedge by selling the stock (positive delta). So the stock price you should use to calculate the price at which you are willing to buy a call should be the price at which you can sell the stock (bid price).
    If you sell a call, you hedge by buying the stock (positive delta). So the stock price you should use to calculate the price at which you are willing to sell a call should be the price at which you can buy the stock (ask price).

For a put the situation is reversed (if you buy a put, you hedge by buying the stock (negative delta)).


Use the correct bid and ask of the stock PriceBook to calculate your bid and ask for the option.

input: Option, PriceBook, r, sigma

output: option bid price, option ask price


In [153]:
def option_quotes(option, pb_S, r, sigma):
    
    
    S1 = best_bid_price(pb_S) #buy call = sell stock, sell put = sell stock 
    S2 = best_ask_price(pb_S) #sell call = buy stock, buy put = buy stock
    #option_p = option_price(option, S, r, sigma)
    if option.kind == 'call':
        K = option.strike
        T = option.tte
        bid_price1 = bs.call_value(S1, K, T, r, sigma)
        ask_price1 = bs.call_value(S2, K, T, r, sigma)
        return bid_price1, ask_price1
    
    if option.kind == 'put':
        K = option.strike
        T = option.tte
        bid_price2 = bs.put_value(S2, K, T, r, sigma)
        ask_price2 = bs.put_value(S1, K, T, r, sigma)
        return bid_price2, ask_price2
    
option_quotes(call, pb_S, r, sigma)

(7.435711264888674, 8.515267758969337)